# RNN

### Time Series

* 일정 시간 간격으로 배치된 데이터들의 수열
* 과거 데이터가 현재/미래에 영향을 미침 => 시계열 데이터를 보고 앞으로 일어날 일을 예측
* 시간을 고려함
* 주가 예측, 날씨 예측, 자연어 처리, 비디오 다음 frame 예측 등

### Sequence Model

* 모델의 입력이나 출력이 Sequence Data인 상황을 다루는 모델
  * sequential data : 텍스트 스트림, 시계열 데이터 등 => 음성 인식, 감정 분석 등
  * <img src="../../Images/Deep_Learning/RNN_1.JPG" width="600" height="300" title=""/>
  * <img src="../../Images/Deep_Learning/RNN_2.JPG" width="600" height="250" title=""/>

### RNN

* <img src="../../Images/Deep_Learning/RNN_3.JPG" width="600" height="250" title=""/>
* 일반 Neural Network vs RNN
  * <img src="../../Images/Deep_Learning/RNN_4.JPG" width="600" height="250" title=""/>
  * 이전 hidden layer의 w, input과의 w, output과 관련된 w => t-1시점의 데이터 사용
* RNN 예시
  * 첫 y를 보면 3번째 값이 가장 크므로 l이 나올 가능성이 높으므로 loss 발생
  * <img src="../../Images/Deep_Learning/RNN_5.JPG" width="600" height="250" title=""/>
* BPTT
  * 역전파 수행 => 이전 데이터를 사용하기 때문
  * <img src="../../Images/Deep_Learning/RNN_6.JPG" width="600" height="250" title=""/>

* RNN 단점
  * 아래와 같은 단점으로 잘 사용되지 않음 => LSTM을 주로 사용
  * <img src="../../Images/Deep_Learning/RNN_7.JPG" width="600" height="250" title=""/>

# LSTM

### LSTM : Long Short Term Memory Network

* 기본 RNN 구조
  * <img src="../../Images/Deep_Learning/RNN_8.JPG" width="600" height="250" title=""/>
* LSTM 구조
  * <img src="../../Images/Deep_Learning/RNN_9.JPG" width="600" height="300" title=""/>
  * 시그마(시그모이드)를 거쳐 이전의 스테이트가 곱해짐
  * 아래 : 히든 스테이트, 위 : 셀 스테이트
  * Forget Gate : 얼마나 정보를 잃을지 / Input Gate : 얼마나 정보를 받을지 / Output Gate : 얼마나 정보를 내보낼지 

* `Cell State`
  * <img src="../../Images/Deep_Learning/RNN_10.JPG" width="600" height="300" title=""/>
  * 멀리 있는 정보도 잘 흐르게 하기 위함 => LSTM 핵심 아이디어

* Forget Gate
  * <img src="../../Images/Deep_Learning/RNN_11.JPG" width="600" height="300" title=""/>
  * 과거 정보를 얼마나 버릴지 => 0에 가까울수록 많이 잃음

* Input Gate
  * <img src="../../Images/Deep_Learning/RNN_12.JPG" width="600" height="300" title=""/>
  * 현재 정보를 얼마나 받을지 => $i_t$
  * 1에 가까울수록 얼마나 더 기억할 것인가의 확률

* Cell State Update
  * <img src="../../Images/Deep_Learning/RNN_13.JPG" width="600" height="300" title=""/>

* Output Gate
  * <img src="../../Images/Deep_Learning/RNN_14.JPG" width="600" height="300" title=""/>
  * 다음 state에 보낼 hidden state


In [1]:
import torch
import torch.nn as nn
import torchvision
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

-  Data : [1,2,3,4,5,6,7,8,9,10,11,12]
- sequence length 3이며 batch size 4인 데이터로 나눔
- [[1,2,3],
   [4,5,6],
   [7,8,9],
   [10,11,12]]

In [2]:
inputs = torch.Tensor([1,2,3,4,5,6,7,8,9,10,11,12])

In [3]:
input_size = 1
seq_length = 3
hidden_size = 2
num_layers = 2
batch_size = 4

### nn.RNN Basic

- input_size: Input의 사이즈에 해당 하는 수를 입력하면 됩니다.
- hidden_size: 은닉층의 사이즈에 해당 하는 수를 입력하면 됩니다.
- num_layers: RNN의 은닉층 레이어 갯수를 나타냅니다. 기본 값은 1입니다.
- bias: 바이어스 값 활성화 여부를 선택합니다. 기본 값은 True 입니다.
- batch_first: True일 시, Output 값의 사이즈는 (batch, seq, feature) 가 됩니다. 기본 값은 False 입니다.
- dropout: 드롭아웃 비율을 설정 합니다. 기본 값은 0입니다.
- bidirectional: True일 시, 양방향 RNN이 됩니다. 기본 값은 False 입니다.

Input : input과 hidden_0 이라는 2개의 input을 받음
- input : neural network로 들어가는 sequence input => [seq_length, batch size, input size]
- hidden_0 : network의 초기 hidden state => [num layers*num directions, batch size, input size] 
    - num directions : 방향(앞->뒤, 뒤->앞), Bidirectional RNN일 경우 2(좌우 방향 모두 고려), 나머지 1
    - hidden_0은 따로 초기화 하지 않으면 Pytorch에 의해 자동으로 모두 0으로 초기화 됨

Output : out과 hidden이라는 2개의 출력을 냄
- out : 마지막 RNN layer로부터 매 timesteps마다의 output
- h_n : 모든 RNN layer로부터 마지막 timestep의 hidden value
    - (num_layers* num_directions, batch, hidden_size)

In [4]:
rnn = nn.RNN(input_size=input_size, hidden_size=hidden_size, num_layers=num_layers, batch_first=True)
# batch_first = True이면 (seq, batch, feature) -> (batch, seq, feature)로 바뀜

In [5]:
# Input
# hidden_0 : 따로 안해서 0으로 초기화

print('inputs before :',inputs.shape)
inputs = inputs.view(batch_size, seq_length, input_size)
print('-'*40)
print('inputs after :',inputs.shape) # [batch size, seq length, input size] => RNN의 input

inputs before : torch.Size([12])
----------------------------------------
inputs after : torch.Size([4, 3, 1])


In [6]:
# Output

out, hidden = rnn(inputs)
print('out:',out.shape) # [batch size, seq length, num_directions*hidden size]
print('hidden:',hidden.shape) # [num layers*num directions, batch size, hidden size]

out: torch.Size([4, 3, 2])
hidden: torch.Size([2, 4, 2])


### Bi-directional RNN

- 시간 순, 시간 역행 순 모두 RNN 적용

In [7]:
rnn = nn.RNN(input_size=input_size, hidden_size=hidden_size, num_layers=num_layers, batch_first=True, bidirectional=True)

In [8]:
# Input

print('inputs :',inputs.shape) # [batch size, seq length, input size]

inputs : torch.Size([4, 3, 1])


In [9]:
# Output
# out : 마지막 RNN layer로부터 매 timesteps마다의 output
# h_n : 모든 RNN layer로부터 마지막 timestep의 hidden value (num_layers* num_directions, batch, hidden_size)

out, hidden = rnn(inputs)
print('out:',out.shape) # [batch size, seq length, num_directions*hidden size]
print('hidden:',hidden.shape) # [num layers*num directions, batch size, hidden size]

out: torch.Size([4, 3, 4])
hidden: torch.Size([4, 4, 2])


### Bi-directional RNN 방향 분리

out

In [10]:
print('out before:',out.shape)
out = out.view(batch_size, seq_length, 2, hidden_size) # 2: 방향이 앞, 뒤로 나눠짐
print('-'*40)
print('out after:',out.shape)

out before: torch.Size([4, 3, 4])
----------------------------------------
out after: torch.Size([4, 3, 2, 2])


* 방향에 따라서만 분리

In [11]:
out_direc1 = out[:,:,0,:]
print('out_direc1:',out_direc1.shape)
out_direc2 = out[:,:,1,:]
print('out_direc2:',out_direc2.shape)

out_direc1: torch.Size([4, 3, 2])
out_direc2: torch.Size([4, 3, 2])


h_n

In [12]:
print('hidden before:', hidden.shape) # 4 = num_layers * num_directions
hidden = hidden.view(num_layers, 2, batch_size, hidden_size) # 2: directions 수
print('-'*40)
print('hidden after:', hidden.shape)

hidden before: torch.Size([4, 4, 2])
----------------------------------------
hidden after: torch.Size([2, 2, 4, 2])


In [13]:
hidden_direc1 = hidden[:,0,:,:]
print('h_n_direc1:',hidden_direc1.shape)
hidden_direc2 = hidden[:,1,:,:]
print('hidden_direc2:',hidden_direc2.shape)

h_n_direc1: torch.Size([2, 4, 2])
hidden_direc2: torch.Size([2, 4, 2])


### RNN application

In [14]:
import torch
import torch.nn as nn
import torchvision
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

# MNIST 데이터셋 
train_data = datasets.MNIST(
    root="../data",
    train=True,
    download=True,
    transform=transforms.ToTensor(),
)

test_data = datasets.MNIST(
    root="../data",
    train=False,
    download=True,
    transform=transforms.ToTensor(),
)

# Data loader
train_loader = DataLoader(train_data, batch_size=128, shuffle=True)
test_loader = DataLoader(test_data, batch_size=128, shuffle=False)

In [15]:
# RNN - many to one
class RNN(nn.Module):
    def __init__(self, num_classes):
        super(RNN, self).__init__()
        self.input_size = 28
        self.hidden_size = 128
        self.num_layers = 2
        self.RNN = nn.RNN(input_size=self.input_size, hidden_size=self.hidden_size, num_layers=self.num_layers, batch_first=True)
        # batch_first = True이면 (seq, batch, feature) -> (batch, seq, feature)로 바뀜
        self.fc = nn.Linear(self.hidden_size, num_classes)
    
    def forward(self, x):
        out, _ = self.RNN(x)  # out: [mini-batch, seq_length, hidden_size]
        out = self.fc(out[:, -1, :]) # out을 fc에 넣기
        return out

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = RNN(num_classes=10).to(device)

In [16]:
# Loss and optimizer
CELoss = nn.CrossEntropyLoss()
adam_optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [17]:
# Train the model
total_epochs = 3
sequence_length = 28
input_size = 28

print('number of iteration :', len(train_loader))

total_step = len(train_loader)
for epoch in range(total_epochs):
    for i, (images, labels) in enumerate(train_loader):
        images = images.reshape(-1, sequence_length, input_size).to(device)
        labels = labels.to(device)
        
        # Forward pass
        outputs = model(images)
        loss = CELoss(outputs, labels)
        
        # Backward and optimize
        adam_optimizer.zero_grad()
        loss.backward()
        adam_optimizer.step()
        
    print ('Epoch [{}/{}], Loss: {:.4f}'.format(epoch+1, total_epochs, loss.item()))

number of iteration : 469
Epoch [1/3], Loss: 0.3272
Epoch [2/3], Loss: 0.2689
Epoch [3/3], Loss: 0.2316


In [18]:
# 학습이 끝난 후 모델 성능 테스트
# test에서는 back propagation 작업을 하지 않으므로 gradient를 계산하지 않도록 함 - 메모리의 효율성을 위해

model.eval()
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_loader:
        images = images.reshape(-1, sequence_length, input_size).to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    print('Test Accuracy of the model on the 10000 test images: {} %'.format(100 * correct / total)) 

Test Accuracy of the model on the 10000 test images: 95.04 %


In [19]:
# 학습한 모델을 model_RNN.ckpt라는 이름으로 저장
torch.save(model.state_dict(), 'model_RNN.ckpt')

### Bidirectional RNN Application

In [20]:
# Many to one
class Bi_RNN(nn.Module):
    def __init__(self, num_classes):
        super(Bi_RNN, self).__init__()
        self.input_size = 28
        self.hidden_size = 128
        self.num_layers = 2
        self.RNN = nn.RNN(input_size=self.input_size, hidden_size=self.hidden_size, num_layers=self.num_layers, batch_first=True, bidirectional=True) # bidirectional=True만 설정해주면 됨!
        # batch_first = True이면 (seq, batch, feature) -> (batch, seq, feature)로 바뀜
        self.fc = nn.Linear(self.hidden_size*2, num_classes) # Bidrectional RNN의 경우 linear에서 hidden_size * 2 해주면 됨!
    
    def forward(self, x):
        # out, hidden => hidden은 알아서 전달됨
        out, _ = self.RNN(x)  # out: [mini-batch, seq_length, hidden_size]
        out = self.fc(out[:, -1, :])
        return out

model = Bi_RNN(num_classes=10).to(device)

In [21]:
# Loss and optimizer
CELoss = nn.CrossEntropyLoss()
adam_optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [22]:
# Train the model
total_epochs = 3
sequence_length = 28
input_size = 28

print('number of iteration :', len(train_loader))

total_step = len(train_loader)
for epoch in range(total_epochs):
    for i, (images, labels) in enumerate(train_loader):
        images = images.reshape(-1, sequence_length, input_size).to(device)
        labels = labels.to(device)
        
        # Forward pass
        outputs = model(images)
        loss = CELoss(outputs, labels)
        
        # Backward and optimize
        adam_optimizer.zero_grad()
        loss.backward()
        adam_optimizer.step()
        
    print ('Epoch [{}/{}], Loss: {:.4f}'.format(epoch+1, total_epochs, loss.item()))

number of iteration : 469
Epoch [1/3], Loss: 0.3543
Epoch [2/3], Loss: 0.0975
Epoch [3/3], Loss: 0.1836


In [23]:
# 학습이 끝난 후 모델 성능 테스트
# test에서는 back propagation 작업을 하지 않으므로 gradient를 계산하지 않도록 함 - 메모리의 효율성을 위해

model.eval()
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_loader:
        images = images.reshape(-1, sequence_length, input_size).to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    print('Test Accuracy of the model on the 10000 test images: {} %'.format(100 * correct / total)) 

Test Accuracy of the model on the 10000 test images: 95.55 %
